In [4]:
import urllib.response
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
from urllib.request import urlopen
import logging
import requests, zipfile, io, sys, time
#http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2003/Qtr1/log20030101.zip
   

    
    
logger = get_logger()
logger.info("Starting Program")
sys.stdout.write("\rProgress : %d%%" % 0)
time.sleep(1)
sys.stdout.write("\rProgress : %d%%" % 1)
time.sleep(1)

year = 0



while ((year < 2003) or (year > 2016)):
    try:
        year = int(input("\nPlease enter a valid year."))
#         print(year)
    except ValueError:
#         print("Not a valid year\n")
        logger.info("\nNot a valid year")
        
sys.stdout.write("\rProgress : %d%%" % 10)
time.sleep(1)
#Data available only till: Qtr1,2016
url = []

if year == "2016":
    qtr1_url(year)
#Any other year excep for 2016, all four quarters    
else:
    logger.info("Generating Links for log files")
    sys.stdout.write("\rProgress : %d%%" % 15)
    qtr1_url(year)
    sys.stdout.write("\rProgress : %d%%" % 20)
    qtr2_url(year)
    sys.stdout.write("\rProgress : %d%%" % 25)
    qtr3_url(year)
    sys.stdout.write("\rProgress : %d%%" % 30)
    qtr4_url(year)
    
#print(url[5])
#Extract all files into folder
i = 30
logger.info("Downloading and Extracting Log files")
print("\nIt may take some time")
for link in url:
    r = requests.get(link)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path="Files2/"+str(year))
    i+=5
    sys.stdout.write("\rProgress : %d%%" % i)
logger.info("Downloading Completed") 
logger.removeHandler("handler")
logging.shutdown()
sys.stdout.write("\rProgress : %d%%" % 100)
sys.stdout.flush()

Starting Program


Progress : 1%
Please enter a valid year.2003
Progress : 10%

Generating Links for log files


Progress : 30%
It may take some time
Progress : 85%

Progress : 100%

In [1]:
#Generate links foreach quarter
def qtr1_url(year):
    for i in range(1,4):
       url.append("http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/" + str(year) + "/"+"Qtr1/"+"log"+str(year)+"0"+str(i)+"01.zip")
       
def qtr2_url(year):
    for i in range(4,7):
       url.append("http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/" + str(year) + "/"+"Qtr2/"+"log"+str(year)+"0"+str(i)+"01.zip")
def qtr3_url(year):
    for i in range(7,10):
       url.append("http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/" + str(year) + "/"+"Qtr3/"+"log"+str(year)+"0"+str(i)+"01.zip")
      
def qtr4_url(year):
    for i in range(10,13):
       url.append("http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/" + str(year) + "/"+"Qtr4/"+"log"+str(year)+str(i)+"01.zip")
 
     
        
 


In [2]:
def get_logger():
    create_directory("Files2")
    loglevel = logging.INFO            # DEBUG, CRITICAL, WARNING, ERROR
    logger = logging.getLogger("Application_Logs")
    logger2 = logging.getLogger("Application_Logs_Stream")
    if not getattr(logger, 'handler_set', None):
        logger.setLevel(logging.INFO)
#         Logfile handler
        handler = logging.FileHandler('Files2/logs.log')
        handler2 = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        logger.addHandler(handler2)
        logger.setLevel(loglevel)
        logger.handler_set = True
#       Stream Handler
    if not getattr(logger, 'handler_set', None):
        logger2.setLevel(logging.INFO)
        handler2 = logging.StreamHandler()
        handler2.setFormatter(formatter)
        logger2.addHandler(handler2)
        logger2.setLevel(loglevel)
        logger2.handler_set = True
        
    return logger

In [3]:
def create_directory(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)